# Data Ingestion 
This is a notebook on data ingestion

# Import necessary libraries

In [11]:
# import os

# def get_structure(startpath):
#     structure = {}

#     for root, dirs, files in os.walk(startpath):
#         path = root.split(os.sep)
#         subdir = path[-1]
#         parent = structure
#         for folder in path[:-1]:
#             parent = parent[folder]
#         parent[subdir] = {}
#         for file in files:
#             parent[subdir][file] = None

#     return structure

# def print_structure(structure, indent=0):
#     for name, substructure in structure.items():
#         print('  ' * indent + name)
#         if isinstance(substructure, dict):
#             print_structure(substructure, indent + 1)

# startpath = Path('text-summarizer')
# folder_structure = get_structure(startpath)
# print_structure(folder_structure)


In [32]:
!pip install dotmap
!pip install python-box

ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml


In [33]:
!pip install PyYAML
!pip install ensure

In [4]:
%pwd
%cd ..

c:\Users\I585498\text-summarizer


In [1]:
import os
os.chdir("../")
import sys
from dataclasses import dataclass
from pathlib import Path
from src.constants.constants import CONFIG_PATH, PARAMS_PATH
from src.loging import logger
from src.utils.common import get_size, create_directories
from src.utils.common import read_yaml


#sys.path.append(str(Path.cwd().parent / "src"))

In [25]:
from src.utils.common import get_size, create_directories

In [27]:
# Create entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):

        self.config = read_yaml(Path(config_filepath).resolve())
        self.params = read_yaml(Path(params_filepath).resolve())

        create_directories(
            path_to_directories=[
                self.config.data_ingestion.root_dir,
                self.config.data_ingestion.unzip_dir
            ],
            verbose=True
        )

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_url,
            local_data_file=config.local_data_dir,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


    
   

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def unzip_data(self):
        logger.info("Unzipping data...")
        
        # Detect file extension
        file_ext = os.path.splitext(self.config.local_data_file)[1]
        
        if file_ext == ".zip":
            os.system(f"unzip {self.config.local_data_file} -d {self.config.unzip_dir}")
        elif file_ext == ".rar":
            os.system(f"unrar x {self.config.local_data_file} {self.config.unzip_dir}")
        elif file_ext in [".7z", ".7zr"]:
            #os.system(f"7z x {self.config.local_data_file} -o{self.config.unzip_dir}")
            with py7zr.SevenZipFile(self.config.local_data_file, mode='r') as z:
                z.extractall(path=self.config.unzip_dir)
        else:
            logger.error(f"Unsupported file format: {file_ext}")
            return

        logger.info(f"Data unzipped at {self.config.unzip_dir}")

    def get_data_size(self):
        logger.info("Getting data size...")
        size = get_size(Path(self.config.unzip_dir))
        logger.info(f"Data size: {size}")

    def run(self):
        self.download_data()
        self.unzip_data()
        

In [30]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.run()
except Exception as e:
    logger.error(e)
    raise e

[2023-08-31 19:58:47,998]: INFO: common: Successfully read yaml file from C:\Users\I585498\text-summarizer\config\config.yaml]
[2023-08-31 19:58:48,005]: INFO: common: Successfully read yaml file from C:\Users\I585498\text-summarizer\params.yaml]
[2023-08-31 19:58:48,009]: INFO: common: Created directory: data/data_ingestion]
[2023-08-31 19:58:48,014]: INFO: common: Created directory: data/data_ingestion/raw_data]


[2023-08-31 19:58:48,644]: INFO: 3992423910: data/data_ingestion/data.7z download! with following info: 
Connection: close
Content-Length: 2944100
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/octet-stream
ETag: "4450eb63089f56212a41409ae9bcb15b3fb051149cf8d4b895f422d5c86b831c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F69E:13934:2FEFA2:3148A4:64F0D26C
Accept-Ranges: bytes
Date: Thu, 31 Aug 2023 17:58:48 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230096-FRA
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1693504729.501220,VS0,VE8
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: edccbd6406c13ad3d26de2ac5ff36a97a116da2d
Expires: Thu, 31 Aug 2023 18:03:48 GMT
Source-Age: 260

]
[2023-08-31 19:58:48,645]: INFO: 3